In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

if sys.platform == 'win32':
    dir = r"D:\黑马课程\人工智能V3.0（尊享无忧）\00 课程资料\3.阶段三数据处理与统计分析\数据处理与统计分析v3.0视频（第二部分）\资料"
else:
    dir = r'/Users/wangrui/Desktop/00 课程资料/3.阶段三数据处理与统计分析/数据处理与统计分析v3.0视频（第二部分）/资料'
os.chdir(dir)

In [7]:
sheet_names = ['2015', '2016', '2017', '2018', '会员等级']
sheet_datas = [pd.read_excel('data/sales.xlsx', sheet_name=i) for i in sheet_names]

In [8]:
for each_name, each_data in zip(sheet_names, sheet_datas):
    print(f'[data summary for ============={each_name}===============]')
    print('Overview:','\n',each_data.head(4))# 展示数据前4条
    print('DESC:','\n',each_data.describe())# 数据描述性信息
    print('NA records',each_data.isnull().any(axis=1).sum()) # 缺失值记录数
    print('Dtypes',each_data.info()) # 数据类型

[data summary for =============2015===============]
Overview: 
           会员ID         订单号       提交日期    订单金额
0  15278002468  3000304681 2015-01-01   499.0
1  39236378972  3000305791 2015-01-01  2588.0
2  38722039578  3000641787 2015-01-01   498.0
3  11049640063  3000798913 2015-01-01  1572.0
DESC: 
                会员ID           订单号           订单金额
count  3.077400e+04  3.077400e+04   30774.000000
mean   2.918779e+10  4.020414e+09     960.991161
std    1.385333e+10  2.630510e+08    2068.107231
min    2.670000e+02  3.000305e+09       0.500000
25%    1.944122e+10  3.885510e+09      59.000000
50%    3.746545e+10  4.117491e+09     139.000000
75%    3.923593e+10  4.234882e+09     899.000000
max    3.954613e+10  4.282025e+09  111750.000000
NA records 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30774 entries, 0 to 30773
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   会员ID    30774 non-null  int64         
 1  

In [9]:
sheet_datas[0].提交日期.max()

Timestamp('2015-12-31 00:00:00')

In [10]:
##去除缺失值和异常值
for i, each_data in enumerate(sheet_datas[:-1]):
    sheet_datas[i] = each_data.dropna() #丢弃
    sheet_datas[i] = sheet_datas[i][sheet_datas[i]['订单金额'] > 1] # 丢弃订单金额<=1的记录
    sheet_datas[i]['max_year_date'] = each_data['提交日期'].max() # 增加一列最大日期值

In [11]:
sheet_datas[0]

,会员ID,订单号,提交日期,订单金额,max_year_date
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31
...,...,...,...,...,...
30769,39368100847,4281994827,2015-12-31,828.0,2015-12-31
30770,409757,4282010457,2015-12-31,199.0,2015-12-31
30771,38380526114,4282017675,2015-12-31,208.0,2015-12-31
30772,28074988,4282019440,2015-12-31,89.0,2015-12-31


In [12]:
#汇总所有数据
data_merge = pd.concat(sheet_datas[:-1])
data_merge['date_interval'] =data_merge['max_year_date'] - data_merge['提交日期']
data_merge['date_interval'] = data_merge['date_interval'].apply(lambda x: x.days)
data_merge['year'] = data_merge['提交日期'].dt.year
data_merge.head()

,会员ID,订单号,提交日期,订单金额,max_year_date,date_interval,year
0,15278002468,3000304681,2015-01-01,499.0,2015-12-31,364,2015
1,39236378972,3000305791,2015-01-01,2588.0,2015-12-31,364,2015
2,38722039578,3000641787,2015-01-01,498.0,2015-12-31,364,2015
3,11049640063,3000798913,2015-01-01,1572.0,2015-12-31,364,2015
4,35038752292,3000821546,2015-01-01,10.1,2015-12-31,364,2015


In [13]:
data_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202827 entries, 0 to 81348
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   会员ID           202827 non-null  int64         
 1   订单号            202827 non-null  int64         
 2   提交日期           202827 non-null  datetime64[ns]
 3   订单金额           202827 non-null  float64       
 4   max_year_date  202827 non-null  datetime64[ns]
 5   date_interval  202827 non-null  int64         
 6   year           202827 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(4)
memory usage: 12.4 MB


In [17]:
#按会员ID做汇总
rfm_gb = data_merge.groupby(['year', '会员ID'], as_index=False).agg({
    'date_interval': 'min', #计算最近一次订单时间
    '提交日期': 'count', #订单数量
    '订单金额': 'sum' # 订单总金额
})
rfm_gb.columns =  ['year','会员ID','R','F','M']
rfm_gb

,year,会员ID,R,F,M
0,2015,267,197,2,105.0
1,2015,282,251,1,29.7
2,2015,283,340,1,5398.0
3,2015,343,300,1,118.0
4,2015,525,37,3,213.0
...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0
148587,2018,39538034662,189,1,3558.0
148588,2018,39538035729,179,1,3699.0
148589,2018,39545237824,275,1,49.0


In [18]:
rfm_gb.iloc[:,2:].describe().T

,count,mean,std,min,25%,50%,75%,max
R,148591.0,165.524043,101.988472,0.0,79.0,156.0,255.0,365.0
F,148591.0,1.365002,2.626953,1.0,1.0,1.0,1.0,130.0
M,148591.0,1323.741329,3753.906883,1.5,69.0,189.0,1199.0,206251.8


In [19]:
# 定义区间边界
r_bins = [-1,79,255,365] # 注意起始边界小于最小值
f_bins = [0,2,5,130] # 通过和业务部门沟通得到
m_bins = [0,69,1199,206252]

In [20]:
# RFM分箱得分
rfm_gb['r_score'] = pd.cut(rfm_gb.R, r_bins, labels=[i for i in range(len(r_bins) - 1, 0, -1)])
rfm_gb['f_score'] = pd.cut(rfm_gb.F, f_bins, labels=[i + 1 for i in range(len(f_bins) - 1)])
rfm_gb['m_score'] = pd.cut(rfm_gb.M, m_bins, labels=[i + 1 for i in range(len(m_bins) - 1)])

In [21]:
rfm_gb

,year,会员ID,R,F,M,r_score,f_score,m_score
0,2015,267,197,2,105.0,2,1,2
1,2015,282,251,1,29.7,2,1,1
2,2015,283,340,1,5398.0,1,1,3
3,2015,343,300,1,118.0,1,1,2
4,2015,525,37,3,213.0,3,2,2
...,...,...,...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0,1,1,1
148587,2018,39538034662,189,1,3558.0,2,1,3
148588,2018,39538035729,179,1,3699.0,2,1,3
148589,2018,39545237824,275,1,49.0,1,1,1


In [22]:
rfm_gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148591 entries, 0 to 148590
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype   
---  ------   --------------   -----   
 0   year     148591 non-null  int64   
 1   会员ID     148591 non-null  int64   
 2   R        148591 non-null  int64   
 3   F        148591 non-null  int64   
 4   M        148591 non-null  float64 
 5   r_score  148591 non-null  category
 6   f_score  148591 non-null  category
 7   m_score  148591 non-null  category
dtypes: category(3), float64(1), int64(4)
memory usage: 6.1 MB


In [23]:
#计算RFM组合
rfm_gb['r_score'] = rfm_gb['r_score'].astype(np.str)
rfm_gb['f_score'] = rfm_gb['f_score'].astype(np.str)
rfm_gb['m_score'] = rfm_gb['m_score'].astype(np.str)
rfm_gb['rfm_group'] = rfm_gb['r_score'].str.cat(rfm_gb['f_score']).str.cat(
    rfm_gb['m_score'])
rfm_gb

,year,会员ID,R,F,M,r_score,f_score,m_score,rfm_group
0,2015,267,197,2,105.0,2,1,2,212
1,2015,282,251,1,29.7,2,1,1,211
2,2015,283,340,1,5398.0,1,1,3,113
3,2015,343,300,1,118.0,1,1,2,112
4,2015,525,37,3,213.0,3,2,2,322
...,...,...,...,...,...,...,...,...,...
148586,2018,39538034299,272,1,49.0,1,1,1,111
148587,2018,39538034662,189,1,3558.0,2,1,3,213
148588,2018,39538035729,179,1,3699.0,2,1,3,213
148589,2018,39545237824,275,1,49.0,1,1,1,111


In [24]:
rfm_gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148591 entries, 0 to 148590
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   year       148591 non-null  int64  
 1   会员ID       148591 non-null  int64  
 2   R          148591 non-null  int64  
 3   F          148591 non-null  int64  
 4   M          148591 non-null  float64
 5   r_score    148591 non-null  object 
 6   f_score    148591 non-null  object 
 7   m_score    148591 non-null  object 
 8   rfm_group  148591 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 10.2+ MB


In [25]:
path = r'C:\Users\17634\DataspellProjects\ds-project\Stage3__Data_Analysis'
os.chdir(path)
rfm_gb.to_excel('sales_rfm_score1.xlsx')  # 保存数据为Excel

In [31]:
import pymysql
import time
# 数据库信息
config = {'host': 'localhost',
          'user': 'root',  # 用户名
          'password': '123456',  # 密码
          'port': 3306,  # 端口，默认为3306
          'database': 'test',  # 数据库名称
          'charset': 'utf8'  # 字符编码
          }
table_name = 'sales_rfm_score'  # 要写库的表名

In [32]:
conn = pymysql.connect(**config)
with conn:
    with conn.cursor() as cursor:
        #梳理数据
        write_db_data = rfm_gb[['会员ID','r_score','f_score','m_score','rfm_group']] # 主要数据
        timestamp = time.strftime('%Y-%m-%d', time.localtime())
        #写库
        for each_value in write_db_data.values:
            insert_sql = "INSERT INTO `{}` VALUES ('{}',{},{},{},'{}','{}')".format(table_name, *each_value, timestamp)
            cursor.execute(insert_sql)
            conn.commit()

In [33]:
# 图形数据汇总
display_data = rfm_gb.groupby(['rfm_group','year'],as_index=False)['会员ID'].count()
display_data.columns = ['rfm_group','year','number']
display_data['rfm_group'] = display_data['rfm_group'].astype(np.int32)
display_data.head()

,rfm_group,year,number
0,111,2015,2180
1,111,2016,1498
2,111,2017,3169
3,111,2018,2271
4,112,2015,3811


In [34]:
# 显示图形
from pyecharts.charts import Bar3D
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
range_color = ['#313695', '#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#ffffbf',
               '#fee090', '#fdae61', '#f46d43', '#d73027', '#a50026']
range_max = int(display_data['number'].max())
c = (
    Bar3D()#设置了一个3D柱形图对象
    .add(
        "",#标题
        [d.tolist() for d in display_data.values],#数据
        xaxis3d_opts=opts.Axis3DOpts( type_="category",name='分组名称'),#x轴数据类型，名称
        yaxis3d_opts=opts.Axis3DOpts( type_="category",name='年份'),#y轴数据类型，名称
        zaxis3d_opts=opts.Axis3DOpts(type_="value",name='会员数量'),#z轴数据类型，名称
    )
    .set_global_opts(#设置颜色，及不同取值对应的颜色
        visualmap_opts=opts.VisualMapOpts(max_=range_max,range_color=range_color),
        title_opts=opts.TitleOpts(title="RFM分组结果"),#设置标题
    )
)
c.render() #在html中显示


'C:\\Users\\17634\\DataspellProjects\\ds-project\\Stage3__Data_Analysis\\render.html'